In [2]:
! pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 34.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 27.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import glob as gb

In [4]:
DIR = 'airflow/logs/dag_id=app_template/*/*/*'
logs_paths = gb.glob(DIR)

In [10]:
df_logs = pd.DataFrame()
for path in logs_paths:
    df = pd.read_csv(path, sep=' ', skiprows=6, header=None)
    df = df[df[1]=='{helpers.py:110}']
    col_names = {
        0: 'timestamp',
        4: 'tasks',
        5: 'batch_id',
        6: 'processed_%',
        7: 'seconds'
    }
    df = df[col_names.keys()]
    df.rename(columns=col_names, inplace=True)
    task_type = path.split('/')[4].split('=')[1].split('.')[0]
    df['task_type'] = task_type
    run_id = path.split('/')[3].split('=')[1].split('__')[1]
    df['run_id'] = run_id
    df_logs = pd.concat([df_logs, df], ignore_index=True)
df_logs.reset_index(drop=True, inplace=True)
df_logs.sample(10)

,timestamp,tasks,batch_id,processed_%,seconds,task_type,run_id
914,[2025-06-07T15:10:08.019+0000],1-1-1,42,33.6,3.913,transformers,090f1ee9-6141-438c-81d5-c63305d74536
999,[2025-06-07T15:07:31.570+0000],1-1-1,2,1.6,1.673,extractors,090f1ee9-6141-438c-81d5-c63305d74536
1352,[2025-06-07T15:24:24.509+0000],1-1-3,23,56.1,1.876,loaders,f96672eb-3e3b-4aed-a2d4-14396ed2f44d
682,[2025-06-07T15:33:34.750+0000],1-1-4,60,48.0,2.466,extractors,928dde66-7066-4c42-a542-e91f68e05434
982,[2025-06-07T15:14:25.899+0000],1-1-1,110,88.0,2.847,transformers,090f1ee9-6141-438c-81d5-c63305d74536
547,[2025-06-07T15:36:30.534+0000],1-1-4,81,64.8,4.444,transformers,928dde66-7066-4c42-a542-e91f68e05434
1414,[2025-06-07T15:25:07.393+0000],1-1-3,44,35.2,1.761,extractors,f96672eb-3e3b-4aed-a2d4-14396ed2f44d
141,[2025-06-07T15:20:31.868+0000],1-1-2,80,64.0,4.429,transformers,72a89e0c-4311-4a85-a418-0d058e7db8de
1125,[2025-06-07T15:23:39.934+0000],1-1-3,3,7.32,2.085,loaders,f96672eb-3e3b-4aed-a2d4-14396ed2f44d
1396,[2025-06-07T15:24:25.729+0000],1-1-3,26,20.8,1.872,extractors,f96672eb-3e3b-4aed-a2d4-14396ed2f44d
